# PH3406 Homework - Ng Chun Seng

# Question 1:

In [131]:
import numpy as np

data = np.loadtxt("HW_Prob1_data.txt")
sigma_x = data[:,0]
sigma_y = data[:,1]
sigma_z = data[:,2]

Direct Inversion:
* Bloch vector, $\overrightarrow{v}_{DI}$ = $(\langle\sigma_x\rangle,\langle\sigma_y\rangle,\langle\sigma_z\rangle)^T$

In [132]:
average_sigma_x = np.sum(sigma_x)/len(sigma_x)
average_sigma_y = np.sum(sigma_y)/len(sigma_y)
average_sigma_z = np.sum(sigma_z)/len(sigma_z)
v_DI = np.array([average_sigma_x,average_sigma_y,average_sigma_z])


print("v_DI =",v_DI)
print("||v_DI|| = ", np.linalg.norm(v_DI))

v_DI = [ 0.392 -0.336  0.876]
||v_DI|| =  1.0168264355336165


Since ||$\overrightarrow{v}_{DI}$|| > 1, we require Maximum Likelihood Estimation
* We use Kullback-Leibler divergence $\mathcal{D}_{KL}(\overrightarrow{v}_{DI},\overrightarrow{v})$ = $N_{x+}ln\left(\frac{1+v_{DI,x}}{1+v_x}\right)+ N_{x-}ln\left(\frac{1-v_{DI,x}}{1-v_x}\right) + 
N_{y+}ln\left(\frac{1+v_{DI,y}}{1+v_y}\right)+ N_{y-}ln\left(\frac{1-v_{DI,y}}{1-v_y}\right) + N_{z+}ln\left(\frac{1+v_{DI,z}}{1+v_z}\right)+ N_{z-}ln\left(\frac{1-v_{DI,z}}{1-v_z}\right)$ 

* For which, we find a Bloch vector $\overrightarrow{v}_{MLE}$ which minimizes $\mathcal{D}_{KL}$, while ensuring ||$\overrightarrow{v}_{DI}$|| $\le 1$

* $N_{m\pm}$ = Number of states in the "$\pm$" state on $\overrightarrow{m}$ axis

In [133]:
N_x_plus = len(sigma_x[sigma_x==1])
N_x_minus = len(sigma_x[sigma_x==-1])
N_y_plus = len(sigma_y[sigma_y==1])
N_y_minus = len(sigma_y[sigma_y==-1])
N_z_plus = len(sigma_z[sigma_z==1])
N_z_minus = len(sigma_z[sigma_z==-1])

def D_kl(v_x,v_y,v_z):
    D_kl = (N_x_plus*np.log((1+average_sigma_x)/(1+v_x)) + N_x_minus*np.log((1-average_sigma_x)/(1-v_x)) + 
    N_y_plus*np.log((1+average_sigma_y)/(1+v_y)) + N_y_minus*np.log((1-average_sigma_y)/(1-v_y)) +
    N_z_plus*np.log((1+average_sigma_z)/(1+v_z)) + N_z_minus*np.log((1-average_sigma_z)/(1-v_z))
    )
    return D_kl

minimum_D_kl_value = 50         #set an arbitrary threshold value for the search of minimum value
v_MLE = []
for v_x in np.linspace(-0.999999,0.999999,200):         #look for the bloch vector v_MLE which minimizes D_KL while ensuring ||v_MLE||<=1 
    for v_y in np.linspace(-0.999999,0.999999,200):
        for v_z in np.linspace(-0.999999,0.999999,200):
            if D_kl(v_x,v_y,v_z) > minimum_D_kl_value or np.linalg.norm(np.array([v_x,v_y,v_z])) > 1 :
                continue
            minimum_D_kl_value = D_kl(v_x,v_y,v_z)
            v_MLE = np.array([v_x,v_y,v_z])

print("v_MLE = ",v_MLE)
print("||v_DI|| = ", np.linalg.norm(v_MLE))

v_MLE =  [ 0.37688405 -0.3165826   0.86934586]
||v_DI|| =  0.9990136919527991


# Question 2:


Since $\hat{A}_m \hat{\rho}_j \hat{A}_n^\dagger = \sum_k \beta_{jk}^{mn}\hat{\rho}_k$ and we have
* $\hat{A}_0 = \mathbb{I}$ ; $\hat{A}_1 = \mathbb{\sigma_x}$ ; $\hat{A}_2 = \mathbb{\sigma_y}$ ;  $\hat{A}_3 = \mathbb{\sigma_z}$
* $\hat{\rho}_k = |\psi_k\rangle\langle\psi_k|$, where $k\in\{0,1,2,3\}$ form basis set for matrices space

We determine complete matrix of $\beta_{jk}^{mn}$ by first determining $\beta_{jk}^{mn}$ for a particular $\hat{\rho}_k$ basis state before varying $\hat{\rho}_k$ and input states of interest $\hat{\rho}_j$ defined as per 
_Example 5.9_, i.e.: $|\psi_k\rangle\langle\psi_k|$ as we have 
* $\hat{\rho}'_k = \mathcal{E}_1(|\psi_k\rangle\langle\psi_k|)$

In [138]:
from qutip import qeye, sigmax,sigmay,sigmaz

def rho(k):
    matrix = np.zeros(4)
    matrix[k] = 1
    return matrix.reshape((2,2))

def A(m):
    if m == 0:
        return qeye(2).full()
    if m == 1:
        return sigmax().full()
    if m == 2:
        return sigmay().full()
    if m == 3:
        return sigmaz().full()
    else: raise ValueError("Invalid operator")
    
def beta(m,n, input):       #input = input state = rho_j     #beta = 2x2 matrix for each m,n value
    return A(m) @ input @ A(n).conj().T

def beta_mn_k(input,k):
    beta_mn_row = np.zeros(16).tolist()            #compiling row elements of beta_mn via mn indexing for kth index of basis state
    for m in range(4):
        for n in range(4):
            beta_mn = beta(m,n,input)
            beta_mn_row[m*4+n] = beta_mn.reshape(-1)[k]
    return beta_mn_row

def full_beta_matrix():
    full_matrix = []
    for j in range(4):
        for k in range(4):
            full_matrix.append(beta_mn_k(rho(j),k))
    return np.array(full_matrix)


Once we constructed $\beta_{jk}^{mn}$, we find its inverse $(\beta^{-1})_{jk}^{mn}$


In [143]:
beta_inverse = np.linalg.inv(full_beta_matrix())

Next, we act $(\beta^{-1})_{jk}^{mn}$ on $\lambda_{jk}$ vector, where _jk_ indices are compiled in similar indexing fashion as per _mn_ indices, as in prior sections

We thus obtain $\chi_{mn} =  \sum_{jk}(\beta^{-1})_{jk}^{mn} \lambda_{jk}$

* For the case of $\gamma = 0$ :

In [ ]:
gamma = 0
lambda_0 = np.array([[1,0],[0,0]]).reshape((2,2))
lambda_1 = np.array([[0,np.sqrt(1-gamma)],[0,0]]).reshape((2,2))
lambda_2 = np.array([[0,0],[np.sqrt(1-gamma),0]]).reshape((2,2))
lambda_3 = np.array([[gamma,0],[0,1-gamma]]).reshape((2,2))

lambda_vector = (lambda_0.reshape(-1)).tolist() + (lambda_1.reshape(-1)).tolist() + (lambda_2.reshape(-1)).tolist() + (lambda_3.reshape(-1)).tolist() 

chi = beta_inverse @ lambda_vector

chi_matrix = chi.reshape((4,4))     #reshape to dimension of (m,n) = (4,4) as per d^2 x d^2

print(chi_matrix)

[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]


* For the case of $\gamma = 0.15$ :

In [148]:
gamma = 0.15
lambda_0 = np.array([[1,0],[0,0]]).reshape((2,2))
lambda_1 = np.array([[0,np.sqrt(1-gamma)],[0,0]]).reshape((2,2))
lambda_2 = np.array([[0,0],[np.sqrt(1-gamma),0]]).reshape((2,2))
lambda_3 = np.array([[gamma,0],[0,1-gamma]]).reshape((2,2))

lambda_vector = (lambda_0.reshape(-1)).tolist() + (lambda_1.reshape(-1)).tolist() + (lambda_2.reshape(-1)).tolist() + (lambda_3.reshape(-1)).tolist() 

chi = beta_inverse @ lambda_vector

chi_matrix = chi.reshape((4,4))     #reshape to dimension of (m,n) = (4,4) as per d^2 x d^2, for representation

with np.printoptions(threshold=np.inf,linewidth=200):
    print(chi_matrix)

[[0.92347722+0.j     0.        +0.j     0.        +0.j     0.0375    +0.j    ]
 [0.        +0.j     0.0375    +0.j     0.        -0.0375j 0.        +0.j    ]
 [0.        +0.j     0.        +0.0375j 0.0375    +0.j     0.        +0.j    ]
 [0.0375    +0.j     0.        +0.j     0.        +0.j     0.00152278+0.j    ]]
